In [6]:
import pandas as pd
import numpy as np
from helpers import calibrate_train_clfs, load_data, get_baseline, print_results, run_clf

In [7]:
keep_columns =  ['IsTraining','FTR', 'Div', 'Date', 'HomeTeam', 'AwayTeam', 
    'away_away_team_corners_against', 'away_away_team_corners_for',	'away_away_team_goals_against',	'away_away_team_goals_for',	
    'away_away_team_possession',	'away_away_team_shotsoff_against',	'away_away_team_shotsoff_for',	'away_away_team_shotson_against',
    	'away_away_team_shotson_for',	'away_direct_team_corners_against',	'away_direct_team_corners_for',	'away_direct_team_goals_against',
        'away_direct_team_goals_for',	'away_direct_team_possession',	'away_direct_team_shotsoff_against',	'away_direct_team_shotsoff_for',
        'away_direct_team_shotson_against',	'away_direct_team_shotson_for',	'home_direct_team_corners_against',	'home_direct_team_corners_for',	
        'home_direct_team_goals_against',	'home_direct_team_goals_for',	'home_direct_team_possession',	'home_direct_team_shotsoff_against',
        'home_direct_team_shotsoff_for',	'home_direct_team_shotson_against',	'home_direct_team_shotson_for',	
        'home_home_team_corners_against',	'home_home_team_corners_for',	'home_home_team_goals_against',	'home_home_team_goals_for',	
        'home_home_team_possession',	'home_home_team_shotsoff_against',	'home_home_team_shotsoff_for',	'home_home_team_shotson_against', 'home_home_team_shotson_for'
]

drop_columns = ['B365H', 'B365D', 'B365A',
    'AC', 'AF', 'AR', 'AS', 'AST', 'AY', 'BWA', 'BWD', 'BWH', 'Bb1X2', 'BbAH', 'BbAHh', 'BbAv<2.5', 'BbAv>2.5', 'BbAvA', 'BbAvAHA', 
    'BbAvAHH', 'BbAvD', 'BbAvH', 'BbMx<2.5', 'BbMx>2.5', 'BbMxA', 'BbMxAHA', 'BbMxAHH', 'BbMxD', 'BbMxH', 'BbOU', 'FTAG', 'FTHG', 'HC', 'HF', 'HR', 'HS', 'HST', 
    'HTAG', 'HTHG', 'HTR', 'HY', 'IWA', 'IWD', 'IWH', 'LBA', 'LBD', 'LBH', 'PSA', 'PSCA', 'PSCD', 'PSCH', 'PSD', 'PSH', 'Referee', 
    'VCA', 'VCD', 'VCH', 'WHA', 'WHD', 'WHH'
]

X,y = load_data(columns_to_drop=drop_columns, is_training=True)

KEEEP
LENGTH800


C:\Users\martin\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\martin\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [7]:
X.shape

(800, 314)

In [16]:
y.shape

(800,)

In [2]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.layers import Bidirectional, LSTM
from keras.initializers import glorot_normal

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

C:\Users\martin\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [10]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(20,  activation='relu')) # input_dim=314,
    #model.add(Bidirectional(LSTM(256,  return_sequences=True, dropout=0.1, recurrent_dropout=0.1,kernel_initializer=glorot_normal(seed=None)),name = 'BDLSTM1')) #Best = 300,0.25,0.25
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [ ]:
seed = 7
numpy.random.seed(seed)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)

In [46]:
results

array([0.43750001, 0.35000001, 0.51250002, 0.37500001, 0.40000001,
       0.41250001, 0.40000001, 0.45000001, 0.41250001, 0.46250001])

In [8]:
X_train_calibrate, X_test, y_train_calibrate, y_test = train_test_split(X, y, test_size=0.20)
X_train, X_calibrate, y_train, y_calibrate = train_test_split(X_train_calibrate, y_train_calibrate, test_size = 0.3, random_state = 42)